Data Preprocessing 

In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, HashingEncoder

df = pd.read_csv('C:\\Users\\fhuba\\Documents\\7th Semester\\ITCS494\\Project\\archive\\Clean_Dataset.csv')
one_hot = OneHotEncoder()
target = TargetEncoder()
hash = HashingEncoder(n_components=16)


airline_encoded = one_hot.fit_transform(df[["airline"]]).toarray()
airline_labels =  [f"airline_{cat}" for cat in one_hot.categories_[0]]
airline_df = pd.DataFrame(airline_encoded, columns=airline_labels)

source_encoded = one_hot.fit_transform(df[["source_city"]]).toarray()
source_labels = [f"source_{cat}" for cat in one_hot.categories_[0]]
source_df = pd.DataFrame(source_encoded, columns=source_labels)

destination_encoded = one_hot.fit_transform(df[["destination_city"]]).toarray()
destination_labels = [f"destination_{cat}" for cat in one_hot.categories_[0]]
destination_df = pd.DataFrame(destination_encoded, columns=destination_labels)

flight_encoded = hash.fit_transform(df['flight'])

df["departure_time"]= target.fit_transform(df['departure_time'], df['price'])
df["arrival_time"]= target.fit_transform(df['arrival_time'], df['price'])

df['stops'] = df['stops'].map({'zero':0, 'one':1, 'two_or_more':2})
df['class'] = df['class'].map({'Economy':0, 'Business':1})

new_df = pd.concat([df,flight_encoded, airline_df, source_df, destination_df], axis=1)
new_df = new_df.drop(["Unnamed: 0", "flight", "airline", "source_city", "destination_city"], axis=1)

print(new_df)


        departure_time  stops  arrival_time  class  duration  days_left  \
0         21232.361894      0  21586.758341      0      2.17          1   
1         20370.676718      0  22231.076098      0      2.33          1   
2         20370.676718      0  14993.139521      0      2.17          1   
3         21630.760254      0  18494.598993      0      2.25          1   
4         21630.760254      0  22231.076098      0      2.33          1   
...                ...    ...           ...    ...       ...        ...   
300148    21630.760254      1  23044.371615      1     10.08         49   
300149    18179.203331      1  21586.758341      1     10.42         49   
300150    20370.676718      1  21586.758341      1     13.83         49   
300151    20370.676718      1  23044.371615      1     10.00         49   
300152    21630.760254      1  23044.371615      1     10.08         49   

        price  col_0  col_1  col_2  ...  source_Delhi  source_Hyderabad  \
0        5953      0    

Random Selection

In [30]:
import random

# Create separate lists of columns based on the DataFrames
airline_cols = airline_df.columns.tolist()
source_cols = source_df.columns.tolist()
destination_cols = destination_df.columns.tolist()
flight_cols = flight_encoded.columns.tolist()
df_cols = new_df.columns.tolist()

# Ensure that the selected columns are from different DataFrames with the new constraint
def random_feature_selection(df_cols, airline_cols, source_cols, destination_cols, flight_cols, num_features):
    selected_cols = []
    remaining_airline_cols = airline_cols.copy()
    remaining_source_cols = source_cols.copy()
    remaining_destination_cols = destination_cols.copy()
    remaining_flight_cols = flight_cols.copy()
    remaining_df_cols = df_cols.copy()  # Track remaining columns from df_cols
    
    # Flags to ensure only one column is selected from each of the specific lists
    airline_selected = False
    source_selected = False
    destination_selected = False
    flight_selected = False
    
    for _ in range(num_features):
        # List of available options
        remaining_dfs = []
        
        # If not already selected, allow choosing a column from the corresponding DataFrame
        if not airline_selected and remaining_airline_cols:
            remaining_dfs.append('airline')
        if not source_selected and remaining_source_cols:
            remaining_dfs.append('source')
        if not destination_selected and remaining_destination_cols:
            remaining_dfs.append('destination')
        if not flight_selected and remaining_flight_cols:
            remaining_dfs.append('flight')
        
        # Always allow selecting from df_cols (no restriction on the number of selections)
        remaining_dfs.append('df_cols')

        # Randomly choose a DataFrame to pick a column from
        chosen_df = random.choice(remaining_dfs)
        
        # Based on the chosen DataFrame, pick a random column and remove it from the list
        if chosen_df == 'airline':
            col = random.choice(remaining_airline_cols)
            remaining_airline_cols.remove(col)
            airline_selected = True  # Flag to prevent selecting again from this list
        elif chosen_df == 'source':
            col = random.choice(remaining_source_cols)
            remaining_source_cols.remove(col)
            source_selected = True  # Flag to prevent selecting again from this list
        elif chosen_df == 'destination':
            col = random.choice(remaining_destination_cols)
            remaining_destination_cols.remove(col)
            destination_selected = True  # Flag to prevent selecting again from this list
        elif chosen_df == 'flight':
            col = random.choice(remaining_flight_cols)
            remaining_flight_cols.remove(col)
            flight_selected = True  # Flag to prevent selecting again from this list
        elif chosen_df == 'df_cols':
            col = random.choice(remaining_df_cols)
            remaining_df_cols.remove(col)
        
        # Add the column to the selected list
        selected_cols.append(col)
        
        # If any of the lists have been exhausted, exclude them from future selections
        if not remaining_airline_cols:
            airline_selected = True
        if not remaining_source_cols:
            source_selected = True
        if not remaining_destination_cols:
            destination_selected = True
        if not remaining_flight_cols:
            flight_selected = True
    
    return selected_cols

# Example: Select 5 random columns
selected_columns = random_feature_selection(df_cols, airline_cols, source_cols, destination_cols, flight_cols, 5)
print("Selected Columns:", selected_columns)


Selected Columns: ['col_1', 'source_Bangalore', 'destination_Hyderabad', 'destination_Kolkata', 'destination_Delhi']
